# Residual Network (ResNet)
---

In [1]:
import tensorflow as tf
import numpy as np

## Hyperparameters

In [3]:
EPOCHS = 10

## Residual Unit

In [4]:
class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        # no activation
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size,
                                            padding = 'same')
        
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size,
                                            padding = 'same')
        
        if filter_in == filter_out:
            self.identity = lambda x: x
        else:
            self.identity = tf.keras.layers.Conv2D(filter_out, (1, 1),
                                                  padding = 'same')
        
    def call(self, x, training = False, mask = None):
        h = self.bn1(x, training = training)
        h = tf.nn.relu(h)
        h = self.conv1(h)
        
        h = self.bn2(h, training = training)
        h = tf.nn.relu(h)
        h = self.conv2(h)
        return self.identity(x) + h        

## Residual Layer

In [5]:
class ResnetLayer(tf.keras.Model):
    def __init__(self, filter_in, filters, kernel_size):
        super(ResnetLayer, self).__init__()
        self.sequence = list()
        # [a] + [b, b, b] -> [a, b, b, b]
        # for f_in, f_out zip([a, b, b, b], [b, b, b]):
        #     the first iteration: f_in, f_out -> a, b
        #     the second iteration: f_in, f_out -> b, b
        #     the third iteration: f_in, f_out -> b, b (the end of the iteration)
        for f_in, f_out in zip([filter_in] + list(filters), filters):
            self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))
        
    def call(self, x, training = False, mask = None):
        for unit in self.sequence:
            # 'training = training' due to Batch Normalization in Residual Unit
            x = unit(x, training = training)
        return x

## Model Definition

In [6]:
class ResNet(tf.keras.Model):
    def __init__(self):
        super(ResNet, self).__init__()
        # 28 X 28 X 8
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding = 'same',
                                          activation = 'relu')
        # 28 X 28 X 16
        self.res1 = ResnetLayer(8, (16, 16), (3, 3))
        # 14 X 14 X 16
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))
        
        # 14 X 14 X 32
        self.res2 = ResnetLayer(16, (32, 32), (3, 3))
        # 7 X 7 X 32
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))
        
        # 7 X 7 X 64
        self.res3 = ResnetLayer(32, (64, 64), (3, 3))
        
        self.flattened = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation = 'relu')
        self.dense2 = tf.keras.layers.Dense(10, activation = 'softmax')
        
    def call(self, x, training = False, mask = None):
        x = self.conv1(x)
        
        x = self.res1(x, training = training)
        x = self.pool1(x)
        x = self.res2(x, training = training)
        x = self.pool2(x)
        x = self.res3(x, training = training)
        
        x = self.flattened(x)
        x = self.dense1(x)
        return self.dense2(x)

## Training and Test Loops

In [7]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training = True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy):
    predictions = model(test_images, training = False)
    t_loss = loss_object(test_labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(test_labels, predictions)

## Dataset

In [8]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Training Environment

In [9]:
# Creating a model
model = ResNet()

# Loss Function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

# Optimization Algorithm
optimizer = tf.keras.optimizers.Adam()

# Evaluation Metrics
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')
test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

## Training

In [10]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer,
                   train_loss, train_accuracy)
        
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object,
                  test_loss, test_accuracy)
        
    print(f'Epoch {epoch + 1}, ' +
         f'Loss: {train_loss.result()}, ' +
         f'Accuracy: {train_accuracy.result() * 100}, ' +
         f'Test Loss: {test_loss.result()}, ' +
         f'Test Accuracy: {test_accuracy.result() * 100}')
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.14129793643951416, Accuracy: 95.95333099365234, Test Loss: 0.12351010739803314, Test Accuracy: 97.3699951171875
Epoch 2, Loss: 0.06706684082746506, Accuracy: 98.15999603271484, Test Loss: 0.06344202905893326, Test Accuracy: 98.0999984741211
Epoch 3, Loss: 0.04806279391050339, Accuracy: 98.63333129882812, Test Loss: 0.032933108508586884, Test Accuracy: 99.06999969482422
Epoch 4, Loss: 0.04380128160119057, Accuracy: 98.81500244140625, Test Loss: 0.033341001719236374, Test Accuracy: 98.98999786376953
Epoch 5, Loss: 0.03269747272133827, Accuracy: 99.0566635131836, Test Loss: 0.033529091626405716, Test Accuracy: 99.0999984741211
Epoch 6, Loss: 0.02923729456961155, Accuracy: 99.19166564941406, Test Loss: 0.0588400661945343, Test Accuracy: 98.3499984741211
Epoch 7, Loss: 0.02437620237469673, Accuracy: 99.30500030517578, Test Loss: 0.04999667778611183, Test Accuracy: 98.94000244140625
Epoch 8, Loss: 0.023697517812252045, Accuracy: 99.3316650390625, Test Loss: 0.067817904055118